# Quarter-Mile Race from Standing Start: Pick Your Cars
This notebook simulates a standing-start quarter-mile race (402.336 m) between two cars.
## How to use
- Run all cells once; an interactive form appears automatically.
- Select a preset for **Car 1** and **Car 2** from the dropdowns (each preset has a
  meaningful name independent of its powertrain).
- Tweak mass, tire compound / width, and (for ICE cars) launch RPM, shift RPM, and
  shift time using the sliders.
- Press **▶  Run Race** to re-run the simulation and update all charts.
## What is configurable per car
- Powertrain type (`ICE`, `BEV`), drivetrain (`RWD` or `AWD`), and tires are set by the
  preset; you can further adjust mass and tire spec in the form.
- ICE cars expose launch RPM, shift RPM, and manual shift time.
- The simulation uses forward-Euler integration with traction-limited launch, aero drag,
  and rolling resistance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import simulation logic from the reusable module.
# quarter_mile_sim.py lives alongside this notebook, which makes it
# importable in CPython, JupyterLab, and JupyterLite (Pyodide / WASM).
from quarter_mile_sim import (
    make_car,
    simulate_quarter_mile,
    QUARTER_MILE_M,
    DEFAULT_DT,
)

# Import UI/plotting helpers (form builder, plots, summary printer).
from quarter_mile_ui import make_car_form, run_race_output

quarter_mile_m = QUARTER_MILE_M
dt = DEFAULT_DT


In [ ]:
# ── Car database ─────────────────────────────────────────────────────────────
# Car names describe character/body style, not powertrain config.
# Each entry follows the Car → Powertrain → Motor / Gearbox schema used by make_car().

CAR_DATABASE = {
    "Sport Coupe": {
        "name": "Sport Coupe",
        "vehicle": {
            "mass": 1500, "CdA": 0.66, "wheel_radius_m": 0.335,
            "rolling_resistance": 0.015,
            "tire": {"width_mm": 300, "compound": "summer", "base_mu": 1.10},
        },
        "powertrain": {
            "type": "ICE", "driving_axles": "RWD",
            "efficiency": {"engine": 1.0, "driveline": 0.90},
            "motors": [{"name": "ICE Engine", "min_rpm": 900, "max_rpm": 7200,
                "torque_curve_rpm_nm": [[1000,380],[2000,570],[3000,860],[4000,850],
                    [5000,850],[6000,800],[7000,690],[7300,650],[8000,100]]}],
            "gearbox": {"type": "manual", "gear_ratios": [3.10,2.10,1.55,1.22,1.00,0.82],
                "final_drive": 3.73, "launch_rpm": 2800, "shift_rpm": 6900, "shift_time_s": 0.30},
        },
    },
    "Electric Sedan": {
        "name": "Electric Sedan",
        "vehicle": {
            "mass": 2400, "CdA": 0.68, "wheel_radius_m": 0.350,
            "rolling_resistance": 0.016,
            "tire": {"width_mm": 300, "compound": "summer", "base_mu": 1.05},
        },
        "powertrain": {
            "type": "BEV", "driving_axles": "AWD",
            "efficiency": {"motor": 1.0, "inverter": 0.96},
            "motors": [{"name": "Combined eMotors", "min_rpm": 0, "max_rpm": 16000,
                "torque_curve_rpm_nm": [[0,850],[6000,850],[7000,810],[8000,660],
                    [10000,520],[12000,420],[14000,340],[16000,290]]}],
            "gearbox": {"type": "single_speed", "ratio": 9.0},
        },
    },
    "Muscle Car": {
        "name": "Muscle Car",
        "vehicle": {
            "mass": 1900, "CdA": 0.72, "wheel_radius_m": 0.345,
            "rolling_resistance": 0.016,
            "tire": {"width_mm": 315, "compound": "summer", "base_mu": 1.08},
        },
        "powertrain": {
            "type": "ICE", "driving_axles": "RWD",
            "efficiency": {"engine": 1.0, "driveline": 0.88},
            "motors": [{"name": "V8 Engine", "min_rpm": 700, "max_rpm": 6500,
                "torque_curve_rpm_nm": [[800,550],[1500,700],[2500,820],[3500,850],
                    [4500,830],[5500,760],[6500,620],[7000,400]]}],
            "gearbox": {"type": "auto",
                "gear_ratios": [4.17,2.34,1.52,1.14,0.87,0.69,0.55,0.46],
                "final_drive": 3.31, "launch_rpm": 1800, "shift_rpm": 6200, "shift_time_s": 0.10},
        },
    },
    "Electric Hypercar": {
        "name": "Electric Hypercar",
        "vehicle": {
            "mass": 1350, "CdA": 0.55, "wheel_radius_m": 0.340,
            "rolling_resistance": 0.014,
            "tire": {"width_mm": 325, "compound": "track", "base_mu": 1.15},
        },
        "powertrain": {
            "type": "BEV", "driving_axles": "AWD",
            "efficiency": {"motor": 1.0, "inverter": 0.97},
            "motors": [{"name": "Dual Motors", "min_rpm": 0, "max_rpm": 20000,
                "torque_curve_rpm_nm": [[0,1250],[5000,1250],[8000,1050],[11000,780],
                    [14000,580],[17000,420],[20000,300]]}],
            "gearbox": {"type": "single_speed", "ratio": 10.5},
        },
    },
    "Performance Wagon": {
        "name": "Performance Wagon",
        "vehicle": {
            "mass": 1780, "CdA": 0.70, "wheel_radius_m": 0.340,
            "rolling_resistance": 0.015,
            "tire": {"width_mm": 295, "compound": "summer", "base_mu": 1.07},
        },
        "powertrain": {
            "type": "ICE", "driving_axles": "AWD",
            "efficiency": {"engine": 1.0, "driveline": 0.88},
            "motors": [{"name": "Turbocharged Engine", "min_rpm": 800, "max_rpm": 6800,
                "torque_curve_rpm_nm": [[1000,420],[1800,600],[2500,680],[3500,670],
                    [4500,650],[5500,610],[6500,540],[7000,380]]}],
            "gearbox": {"type": "auto",
                "gear_ratios": [3.91,2.29,1.55,1.16,0.86,0.73],
                "final_drive": 3.55, "launch_rpm": 2200, "shift_rpm": 6500, "shift_time_s": 0.08},
        },
    },
}

In [ ]:
import ipywidgets as widgets
from IPython.display import display

_DB_KEYS = list(CAR_DATABASE.keys())

form1, get_spec1 = make_car_form("Car 1", _DB_KEYS[0], CAR_DATABASE)
form2, get_spec2 = make_car_form("Car 2", _DB_KEYS[1], CAR_DATABASE)

run_btn = widgets.Button(
    description="▶  Run Race", button_style="success",
    layout=widgets.Layout(width="150px", margin="8px 0"),
)
out = widgets.Output()


def _run_race(_=None):
    name1, spec1 = get_spec1()
    name2, spec2 = get_spec2()
    cars    = {name1: make_car(name1, spec1), name2: make_car(name2, spec2)}
    results = {n: simulate_quarter_mile(c, dt=dt) for n, c in cars.items()}
    with out:
        out.clear_output(wait=True)
        run_race_output(cars, results)


run_btn.on_click(_run_race)
display(widgets.HBox([form1, form2]))
display(run_btn)
display(out)
_run_race()  # auto-run on cell execution


## Notes
- Select presets from the dropdown; car names describe character (e.g. *Sport Coupe*,
  *Muscle Car*), not drivetrain type.
- Adjust mass, tire compound/width, and (for ICE) launch/shift RPM in the form, then
  press **▶  Run Race**.
- `powertrain.driving_axles` (`RWD`/`AWD`) and `powertrain.type` (`ICE`/`BEV`) are
  set by the preset and shown as a read-only label next to the preset dropdown.
- Manual shift (`shift_time_s`) introduces a zero-wheel-power window during each shift.
- Wheel power includes efficiency losses and is always lower than source power.
- To add a custom preset, extend `CAR_DATABASE` in the cell above the form.
## Code organisation
| File | Responsibility |
|------|---------------|
| `quarter_mile_sim.py` | Physics engine: `make_car`, `simulate_quarter_mile`, helpers |
| `quarter_mile_ui.py`  | Widgets form builder (`make_car_form`) and all plot / summary functions |
| `quarter_mile_race.ipynb` | Car database, wiring, and interactive display only |
